In [ ]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [ ]:
class Module(object):
    
    def __init__ (self):
        self.output = None
        self.mode='Train'
        
        self.gradInput = None
    """
    Basically, you can think of a module as of a something (black box) 
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`: 
        
        output = module.forward(input)
    
    The module should be able to perform a backward pass: to differentiate the `forward` function. 
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule. 
    
        gradInput = module.backward(input, gradOutput)
    """
    
    def forward(self, inpt):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(inpt)

    def backward(self,gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.
        
        This includes 
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(gradOutput)
        self.accGradParameters(gradOutput)
        return self.gradInput
    

    def updateOutput(self, inpt):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.
        
        Make sure to both store the data in `output` field and return it. 
        """
        
        # The easiest case:
            
        # self.output = input 
        # return self.output
        
        pass

    def updateGradInput(self, gradOutput):
        """
        Computing the gradient of the module with respect to its own input. 
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.
        
        The shape of `gradInput` is always the same as the shape of `input`.
        
        Make sure to both store the gradients in `gradInput` field and return it.
        """
        
        # The easiest case:
        
        # self.gradInput = gradOutput 
        # return self.gradInput
        
        pass   
    
    def accGradParameters(self, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass
    
    def zeroGradParameters(self): 
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass
        
    def getParameters(self):
        """
        Returns a list with its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
        
    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
    
    def training(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.mode = 'Train'
    
    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.mode = 'Test'
    
    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return 'Module'

# Sequential container

**Define** a forward and backward pass procedures.

In [ ]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially. 
         
         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`. 
    """
    
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
        self.y = []
        self.loss=None
   
    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)
    def forward(self, X):
        # Call forward method on the input layer
        # this will set the output property that
        # the first layer in "prev" object is expecting
#         self.create_sequential()
 
        prediction= self.updateOutput(X)

        return prediction
    
    def predict(self,X):
        prediction_proba=self.forward(X)
        return np.argmax(prediction_proba,axis=1)
    
  
                  


    def updateOutput(self, inpt):
        
#         self.input_layer.forward(inpt)
        # Call forward method of every object in a chain
        # Pass output of the previous object as a parameter
        output=inpt
        for layer in self.modules:
            output=layer.forward(output)
            
        self.output=output  
            
        return self.output

    def backward(self,gradOutput):
        
        gradInput=gradOutput
        for layer in reversed(self.modules):
            gradInput=layer.backward(gradInput)
        self.gradInput=gradInput    

        return self.gradInput
    
    def evaluate(self):
        for layer in self.modules:
            layer.evaluate()
    
    def training(self):
        
         for layer in self.modules:
            layer.training()

    
    
    
    
      

    def zeroGradParameters(self): 
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]
    
    def __repr__(self):
        string = ''.join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self,x):
        return self.modules.__getitem__(x)

In [ ]:
class Optimizer_SGD:
# Initialize optimizer - set settings,
# learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=0.1):
        self.learning_rate = learning_rate
    # Update parameters
    def update_params(self, layer):
#         layer.weights += -self.learning_rate * layer.dweights
#         layer.biases += -self.learning_rate * layer.dbiases
       

        layer.W += -self.learning_rate * layer.gradW
        layer.biases += -self.learning_rate * layer.gradb


# Layers

- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [ ]:
class Linear_Input:
    def forward(self,inputs):
        self.output=inputs

In [ ]:
class Linear(Module):
    """
    A module which applies a linear transformation 
    A common name is fully-connected layer, InnerProductLayer in caffe. 
    
    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()
       
        # This is a nice initialization
        stdv = 1. / np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_in, n_out))
        self.biases = np.zeros((1, n_out))
        
        self.gradW = np.zeros_like(self.W)
        self.gradb=np.zeros_like(self.biases)
        
    
    
        
    def updateOutput(self, inpt):
        
        if self.mode=='Train':
            self.inpt=inpt
            
            
        self.output = inpt.dot(self.W)+self.biases
        return self.output

    def updateGradInput(self, gradOutput):
        self.gradInput = gradOutput.dot(self.W.T)
        
        return self.gradInput
    
    def accGradParameters(self, gradOutput):
        self.gradW = self.inpt.T.dot(gradOutput)
        self.gradb = np.sum(gradOutput, axis=0, keepdims=True)
    
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        return [self.W, self.biases]
    
    def getGradParameters(self):
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[0],s[1])
        return q

This one is probably the hardest but as others only takes 5 lines of code in total. 
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [ ]:
class SoftMax(Module):
    def __init__(self):
        super(SoftMax, self).__init__()  
    
    def updateOutput(self, inpt):
        
        probabilities = np.exp(inpt) / np.sum(np.exp(inpt), axis=1, keepdims=True)
        self.output = probabilities
        if self.mode=='Train':
            self.inpt=probabilities
        return self.output
    
    def updateGradInput(self,gradOutput):
        self.gradInput = np.empty_like(gradOutput)
        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in \
        enumerate(zip(self.inpt,gradOutput )):
        # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output and
            jacobian_matrix = np.diagflat(single_output) - \
            np.dot(single_output, single_output.T)
            # Calculate sample-wise gradient
            # and add it to the array of sample gradients
            self.gradInput[index] = np.dot(jacobian_matrix,single_dvalues)
        
        return self.gradInput
    
    def __repr__(self):
        return 'SoftMax'

One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement a part of the layer: mean subtraction. That is, the module should calculate mean value for every feature (every column) and subtract it.

Note, that you need to estimate the mean over the dataset to be able to predict on test examples. The right way is to create a variable which will hold smoothed mean over batches (exponential smoothing works good) and use it when forwarding test examples.

When training calculate mean as folowing: 
```
    mean_to_subtract = self.old_mean * alpha + batch_mean * (1 - alpha)
```
when evaluating (`self.training == False`) set $alpha = 1$.


- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [ ]:
class BatchMeanSubtraction(Module):
    def __init__(self, alpha = 0.95):
        super(BatchMeanSubtraction, self).__init__()
        self.trainable=True
        self.W=None
        self.biases=None
        self.alpha = alpha
        self.old_mean = None
        
        
    def updateOutput(self, inpt):

        if not isinstance(self.W,np.ndarray):
            self.W=np.random.normal(size=(1,inpt.shape[1]))
            self.biases=np.random.normal(size=(1,inpt.shape[1]))
            self.running_mean=np.zeros(inpt.shape[1])
            self.running_var=np.zeros(inpt.shape[1])
            
        if self.mode=='Train':
            self.mu = np.mean(inpt,axis=0)
            self.sd = np.sqrt(np.var(inpt,axis=0))
            self.N = inpt.shape[0]
            normalized = (inpt - self.mu) / self.sd
            self.inpt=normalized
            gammax=self.W*normalized
            self.running_mean=self.running_mean*self.alpha+self.mu*(1-self.alpha)
            self.running_var=self.running_var*self.alpha+self.sd*(1-self.alpha)
            self.output=gammax+self.biases
        elif self.mode=='Test':
            normalized=(inpt-self.running_mean)/self.running_var
            self.output=self.W*normalized+self.biases   
               

        return self.output
    
    def updateGradInput(self,gradOutput):
        N,D=gradOutput.shape
        dxhat=gradOutput*self.W
        dx=(1. / N) * self.sd * (N*dxhat - np.sum(dxhat, axis=0)- self.inpt*np.sum(dxhat*self.inpt, axis=0))
        
        self.gradInput=dx
        
        return self.gradInput
    
    def accGradParameters(self, gradOutput):
        self.gradW = np.sum(self.inpt*gradOutput,axis=0)
        self.gradb = np.sum(gradOutput, axis=0, keepdims=True)
        
    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters. 
        If the module does not have parameters return empty list. 
        """
        return [self.gradW,self.gradb]    
    
    
    def __repr__(self):
        return 'BatchMeanNormalization'

Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. 

This is a very cool regularizer. In fact, when you see your net is overfitting try to add more dropout. It is hard to test, since every `forward` requires sampling a new mask, that is the only reason we need `fix_mask` parameter in there. 

While training (`self.training == True`) it should sample a mask on each iteration (for every batch). When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [ ]:
from scipy.stats import bernoulli

class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        self.p = p
        self.mask = None

    def updateOutput(self, inpt):
        if self.mode=='Train':
            self.mask = bernoulli.rvs(self.p, size=inpt.size).reshape(inpt.shape)
            self.output = inpt * self.mask
        elif self.mode=='Test':
             self.output=inpt
        
        return  self.output
    
    def updateGradInput(self, gradOutput):
        
        self.gradInput = gradOutput * self.mask
        return self.gradInput
    
        
    def __repr__(self):
        return 'Dropout'

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**): 

In [ ]:
class ReLU(Module):
    def __init__(self):
        super(ReLU, self).__init__()
    
    def updateOutput(self, inpt):
        
        if self.mode=='Train':
            self.inpt=inpt
        self.output = np.maximum(inpt, 0)
        return self.output
    
    def updateGradInput(self, gradOutput):
        self.gradInput=gradOutput.copy()
        self.gradInput[self.inpt <= 0]=0

        return self.gradInput
    
    def __repr__(self):
        return 'ReLU'

Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope. 

In [ ]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()   
        self.slope = slope
        
    def updateOutput(self, inpt):
        if self.mode=='Train':
            self.inpt=inpt
        self.output = np.maximum(inpt, self.slope * inpt)
        return self.output
    
    def updateGradInput(self, gradOutput):
        self.gradInput = np.multiply(gradOutput, np.multiply(self.inpt > self.slope, self.inpt))
        return self.gradInput
    
    def __repr__(self):
        return 'LeakyReLU'

Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

In [ ]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        super(ELU, self).__init__() 
        self.alpha = alpha
        
    def updateOutput(self, inpt):
        if self.mode=='Train':
            self.inpt=inpt
        self.output=np.where(inpt>0,inpt,self.alpha*(np.exp(inpt)-1))
        return self.output
    
    def updateGradInput(self,gradOutput):
        
        self.gradInput = np.multiply(gradOutput, np.where(self.inpt>0,1,self.alpha*np.exp(self.inpt)))
        return self.gradInput
    
    def __repr__(self):
        return 'ELU'

Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

In [ ]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()
    
    def updateOutput(self, inpt):
        if self.mode=='Train':
            self.inpt=inpt
        self.output = np.log(np.exp(inpt) + 1)
        return self.output
    
    def updateGradInput(self,gradOutput):
        self.gradInput = np.multiply(gradOutput, np.exp(self.inpt) / (np.exp(self.inpt) + 1))
        return self.gradInput
    
    def __repr__(self):
        return 'SoftPlus'

# Criterions

Criterions are used to score the models answers. 

In [ ]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
    
    def forward(self, inpt, target):
        """
            Given an input and a target, compute the loss function 
            associated to the criterion and return the result.
            
            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(inpt, target)

    def backward(self, inpt, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result. 

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(inpt, target)
    
    def updateOutput(self, inpt, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, inpt, target):
        """
        Function to override.
        """
        return self.gradInput   

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return 'Criterion'

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.

In [ ]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()

    def updateOutput(self, inpt, target):   
        self.output = np.mean((target - inpt)**2, axis=-1)
        
        return self.output 
 
    def updateGradInput(self, inpt, target):
        
        self.gradInput=-2*(target-inpt)/len(inpt[0])/len(inpt)
        return self.gradInput

    def __repr__(self):
        return 'MSECriterion'

You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](https://www.kaggle.com/wiki/MultiClassLogLoss). Nevertheless there is a sum over `y` (target) in that formula, 
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. 

In [ ]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        super(ClassNLLCriterion, self).__init__()
    
    def updateOutput(self, inpt, target):
        
        input_clamp = np.clip(inpt, 1e-7, 1 - 1e-7)
        self.output=(-np.log(np.sum(input_clamp* target,axis=1))).mean()
        return self.output

    def updateGradInput(self, inpt, target):
        
        # Use this trick to avoid numerical errors
        input_clamp = np.maximum(1e-15, np.minimum(inpt, 1 - 1e-15))       
        self.gradInput = -target / input_clamp / inpt.shape[0]
        
        return self.gradInput
    
    def __repr__(self):
        return 'ClassNLLCriterion'

In [ ]:
def one_hot(y):
    one_hot_labels=np.zeros((len(y),len(np.unique(y))))
    for i in range(len(y)):
        one_hot_labels[i,y[i]]=1
    return one_hot_labels